In [170]:
# Проект Дашборд конверсий

In [171]:
## Загрузка библиотек

In [238]:
import pandas as pd
import requests as req

In [239]:
## Чтение и выборка зарегистрированных и посещаемых пользователей

In [240]:
START = '2023-03-01'
END = '2023-09-01'
visitors_req = req.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01')
visitors = pd.DataFrame(visitors_req.json())
regs_req = req.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01')
regs = pd.DataFrame(regs_req.json())
visitors['datetime'] = pd.to_datetime(visitors['datetime'])
regs['datetime'] = pd.to_datetime(regs['datetime'])

In [241]:
regs.head()

,datetime,user_id,email,platform,registration_type
0,2023-03-01 07:40:13,2e0f6bb8-b029-4f45-a786-2b53990d37f1,ebyrd@example.org,web,google
1,2023-03-01 13:14:00,f007f97c-9d8b-48b5-af08-119bb8f6d9b6,knightgerald@example.org,web,email
2,2023-03-01 03:05:50,24ff46ae-32b3-4a74-8f27-7cf0b8f32f15,cherylthompson@example.com,web,apple
3,2023-03-01 00:04:47,3e9914e1-5d73-4c23-b25d-b59a3aeb2b60,halldavid@example.org,web,email
4,2023-03-01 18:31:52,27f875fc-f8ce-4aeb-8722-0ecb283d0760,denise86@example.net,web,google


In [242]:
visitors.head(100)

,visit_id,platform,user_agent,datetime
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01 10:36:22
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,2023-03-01 06:25:00
2,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01 10:26:13
3,19322fed-157c-49c6-b16e-2d5cabeb9592,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2023-03-01 12:33:06
4,04762a22-3c9f-40c9-9ac9-6628c4381836,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01 01:38:35
...,...,...,...,...
95,79cf0607-cb47-4aac-b811-84f95ddbb589,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01 14:34:32
96,e363bad9-23d8-4263-aa84-727e10b07891,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01 13:59:45
97,54ef7f95-9688-4018-a535-7be9e7c5af18,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01 06:41:50
98,c0c16b36-c0ea-42d1-9950-a263faec571d,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01 02:14:30


In [243]:
## Анализ зарегистрированных пользователей

In [244]:
# visitors.describe()

In [245]:
## Анализ посещаемых пользователей

In [246]:
# regs.describe()

In [247]:
## Регистрация уникальных пользователей

In [248]:
visitors = visitors.drop_duplicates(subset="visit_id", keep="last")
visitors = visitors[~visitors['platform'].isin(['bot'])]

In [249]:
## Группировка пользователей

In [250]:
grouped_regs = regs.groupby([regs.datetime.dt.to_period('D'), 'platform'])['datetime'].agg('count')
grouped_regs.head()

datetime    platform
2023-03-01  android     61
            ios         18
            web          8
2023-03-02  android     59
            ios         24
Name: datetime, dtype: int64

In [251]:
grouped_visitors = visitors.groupby([visitors.datetime.dt.to_period('D'), 'platform']).agg('count')

In [252]:
grouped_visitors.head(100)

visit_id  user_agent  datetime
datetime   platform                                
2023-03-01 android         75          75        75
           ios             22          22        22
           web            787         787       787
2023-03-02 android         67          67        67
           ios             31          31        31
...                       ...         ...       ...
2023-04-01 web            729         729       729
2023-04-02 android         69          69        69
           ios             60          60        60
           web            705         705       705
2023-04-03 android         96          96        96

[100 rows x 3 columns]

In [204]:
conversion = pd.merge(
    grouped_visitors, grouped_regs,
    left_on='platform', right_on='platform'
)

In [205]:
conversion.head()

,visit_id,user_agent,datetime,user_id,email,registration_type
platform,,,,,,
android,1,1,61,61,61,61
android,1,1,59,59,59,59
android,1,1,22,22,22,22
android,1,1,77,77,77,77
android,1,1,54,54,54,54
